In [1]:
import polars as pl

## lazy permite que las consultas se ejecuten en forma de streaming collect(streaming=True) se pasa este argumento

In [2]:
query = (
    pl.scan_parquet('/content/vuelos.parquet')
    .filter(pl.col('MONTH')>6)
    .group_by('AIRLINE').agg(
        media_tiempo_aire=pl.col('AIR_TIME').mean()

    )
)

In [3]:
vuelos_agregados= query.collect(streamin=True)

# NO TODAS LAS OPERACIONES LAZY ADMITEN streaming=True

In [4]:
vuelos_agregados

AIRLINE,media_tiempo_aire
str,f64
"""HA""",85.561475
"""NK""",132.105457
"""MQ""",64.836925
"""DL""",115.037619
"""F9""",130.735121
…,…
"""AA""",136.295891
"""B6""",143.185472
"""AS""",158.042161


### streaming esta soportado para las siguientes operaciones:
##filter, head, tail
##with_columns,select
##group_by,join
##unique, sort
##explode
##scan_csv, scan_parquet

In [6]:
print(query.explain(streaming=True))  ## detalla las operaciones en modo streaming

STREAMING:
  AGGREGATE
  	[col("AIR_TIME").mean().alias("media_tiempo_aire")] BY [col("AIRLINE")] FROM
    Parquet SCAN [/content/vuelos.parquet]
    PROJECT 3/31 COLUMNS
    SELECTION: [(col("MONTH")) > (6)]


In [7]:
query_2 = (
    pl.scan_parquet('/content/vuelos.parquet')
    .with_columns(
        pl.col('AIR_TIME').mean().over('AIR_TIME')
    )
)

In [8]:
print(query_2.explain(streaming=True))

 WITH_COLUMNS:
 [col("AIR_TIME").mean().over([col("AIR_TIME")])] 
  STREAMING:
    Parquet SCAN [/content/vuelos.parquet]
    PROJECT */31 COLUMNS
